In [113]:
import pandas as pd
from arcgis import GIS
gis = GIS()
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import requests
import arcpy
import geopandas as gpd

In [114]:
#strRelease = "WF TDM v9.0.1"
lstFieldsFromForecasts = ['SEGID','MF2023','MF2028','MF2032','MF2042','MF2050','ADJ2023','ADJ2028','ADJ2032','ADJ2042','ADJ2050','ADJHIST']

In [135]:
# Your initial DataFrame
dfSegmentFilenames = pd.DataFrame([
    ["F_AREA=='Cache'"                   , 'data/segments/Segments_State_20231221_Draft.shp'                 , True ],
    ["PLANAREA=='WFRC' | PLANAREA=='MAG'", 'data/segments/WFv901_Segments_20240409_GeoKeys_AADTs_Factors.shp', False]
], columns=['filter', 'filename', 'updateRoute'])

# Initialize an empty list to store each filtered GeoDataFrame
_filteredGdfs = []

# Iterate through each row in your DataFrame
for _, row in dfSegmentFilenames.iterrows():
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(row['filename'])
    
    # Use the `query` method to apply the filter from the 'filter' column
    _filteredGdf = gdf.query(row['filter'])
    
    if row['updateRoute']:
        # Assuming 'SEGID' is the correct column to apply this operation
        _filteredGdf.loc[:, 'ROUTE'] = _filteredGdf['SEGID'].apply(lambda x: x.split('_')[0] if '_' in x else x)

    # Add the filtered GeoDataFrame to the list
    _filteredGdfs.append(_filteredGdf)

# Concatenate all filtered GeoDataFrames into a single GeoDataFrame
gdfs = pd.concat(_filteredGdfs, ignore_index=True)

# Assuming you want to display or further process `gdfs`
display(gdfs)


,Id,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,F_AREA,...,SUTRK2014,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010
0,0,0023_000.0,0.000,1.564,1.585102,Cache,1969.0,2.0,5.0,Cache,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0023_001.6,1.564,2.146,0.580084,Cache,2955.0,2.0,5.0,Cache,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0023_002.1,2.146,2.533,0.384494,Cache,2955.0,2.0,5.0,Cache,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0023_002.5,2.533,4.170,1.633041,Cache,2209.0,2.0,5.0,Cache,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0023_004.2,4.170,6.289,2.114815,UDOT,2209.0,2.0,5.0,Cache,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,0,WFRC_8472,0.000,0.000,0.253734,WFRC,0.0,1.0,35.0,NaN,...,0,0,0,0,0.0,0,0,0,0,0
5834,0,WFRC_8473,0.000,0.000,0.714015,WFRC,0.0,1.0,11.0,NaN,...,0,0,0,0,0.0,0,0,0,0,0
5835,0,WFRC_8474,0.000,0.000,0.232025,WFRC,0.0,1.0,35.0,NaN,...,0,0,0,0,0.0,0,0,0,0,0
5836,0,WFRC_8475,0.000,0.000,0.587918,WFRC,0.0,1.0,35.0,NaN,...,0,0,0,0,0.0,0,0,0,0,0


In [136]:
# drop unneeded columns and fill nas
sdfSegments = GeoAccessor.from_geodataframe(gdfs)
sdfSegments.drop(columns=['Id','F_AREA'], inplace=True) # only used for art of forecasting tool
sdfSegments

,SEGID,BMP,EMP,DISTANCE,PLANAREA,AADT2019,SUBAREAID,CO_FIPS,CCSGROUP19,FACMANADJ,...,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010,SHAPE
0,0023_000.0,0.000,1.564,1.585102,Cache,1969.0,2.0,5.0,COS,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[422233.4001000002, 4607564.0001],..."
1,0023_001.6,1.564,2.146,0.580084,Cache,2955.0,2.0,5.0,COS,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[422182.7999999998, 4610094.1], [4..."
2,0023_002.1,2.146,2.533,0.384494,Cache,2955.0,2.0,5.0,COS,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[422217.0, 4611027.0], [422220.400..."
3,0023_002.5,2.533,4.170,1.633041,Cache,2209.0,2.0,5.0,COP,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[421790.7599999998, 4611304.15], [..."
4,0023_004.2,4.170,6.289,2.114815,UDOT,2209.0,2.0,5.0,COP,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{""paths"": [[[420610.13999999966, 4613651.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,0.000,0.000,0.253734,WFRC,0.0,1.0,35.0,COU,0.0,...,0,0,0,0.0,0,0,0,0,0,"{""paths"": [[[424308.5, 4489341.199999999], [42..."
5834,WFRC_8473,0.000,0.000,0.714015,WFRC,0.0,1.0,11.0,COT,0.0,...,0,0,0,0.0,0,0,0,0,0,"{""paths"": [[[417141.7167999996, 4545904.8543],..."
5835,WFRC_8474,0.000,0.000,0.232025,WFRC,0.0,1.0,35.0,COT,0.0,...,0,0,0,0.0,0,0,0,0,0,"{""paths"": [[[422745.79000000004, 4480897.97000..."
5836,WFRC_8475,0.000,0.000,0.587918,WFRC,0.0,1.0,35.0,COT,0.0,...,0,0,0,0.0,0,0,0,0,0,"{""paths"": [[[422006.9967, 4481267.654200001], ..."


In [140]:
sdfSegments.groupby(['PLANAREA']).agg(COUNT=('SEGID','count'))

,COUNT
PLANAREA,
Cache,402
MAG,2097
UDOT,87
WFRC,3252


In [141]:
# Connect to GIS
gis = GIS()

# Access the feature layer
feature_layer_url = 'https://services1.arcgis.com/taguadKoI1XFwivx/arcgis/rest/services/Forecasts_gdb/FeatureServer/0'
feature_layer = FeatureLayer(feature_layer_url)

# Convert to Spatially Enabled DataFrame
sdfForecasts = feature_layer.query().sdf
sdfForecasts = sdfForecasts[lstFieldsFromForecasts] # filter columns by list

# Display the first few rows of the dataframe
display(sdfForecasts)


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
years = [2023, 2028, 2032, 2042, 2050]

for year in years:
    # F  : Forecast 
    # MF : Model Forecast (model volume + base year adjustment)
    # ADJ: Manual Adjustment
    sdfForecasts[f'F{year}'] = sdfForecasts[f'MF{year}'] + sdfForecasts[f'ADJ{year}'] + sdfForecasts['ADJHIST']

sdfForecasts

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,ADJ2050,ADJHIST,F2023,F2028,F2032,F2042,F2050
0,0013_000.0,20000.0,21000.0,22000.0,21500.0,22000.0,0.0,0.0,-500.0,0.0,0.0,0.0,20000.0,21000.0,21500.0,21500.0,22000.0
1,0013_000.6,20000.0,21000.0,21000.0,22000.0,23000.0,0.0,0.0,0.0,0.0,0.0,0.0,20000.0,21000.0,21000.0,22000.0,23000.0
2,0013_001.3,16000.0,17000.0,17000.0,18500.0,19500.0,0.0,0.0,0.0,0.0,0.0,0.0,16000.0,17000.0,17000.0,18500.0,19500.0
3,0013_001.6,17000.0,18000.0,18000.0,19500.0,20000.0,0.0,0.0,0.0,0.0,0.0,0.0,17000.0,18000.0,18000.0,19500.0,20000.0
4,0013_001.8,17000.0,17500.0,17500.0,18500.0,19000.0,0.0,0.0,0.0,0.0,0.0,0.0,17000.0,17500.0,17500.0,18500.0,19000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9631,UDOT_7090,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9632,UDOT_7091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9633,UDOT_7092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9634,UDOT_7093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
sdfForecasts.columns

Index(['SEGID', 'MF2023', 'MF2028', 'MF2032', 'MF2042', 'MF2050', 'ADJ2023',
       'ADJ2028', 'ADJ2032', 'ADJ2042', 'ADJ2050', 'ADJHIST', 'F2023', 'F2028',
       'F2032', 'F2042', 'F2050'],
      dtype='object')

In [144]:
display(sdfSegments.columns)

# Your initial list of fields you consider dropping, excluding 'SEGID'
columns_to_drop = [column for column in lstFieldsFromForecasts if column != 'SEGID']

# Calculate the intersection of columns to drop and existing columns
columns_to_drop_exist = [column for column in columns_to_drop if column in sdfSegments.columns]

# Drop these columns from sdfSegments if they exist
sdfSegments.drop(columns=columns_to_drop_exist, inplace=True)

# Display remaining columns in sdfSegments to verify
display(sdfSegments.columns)


Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'CUTRK2014', 'SUTRK2013', 'CUTRK2013', 'SUTRK2012', 'CUTRK2012',
       'SUTRK2011', 'CUTRK2011', 'SUTRK2010', 'CUTRK2010', 'SHAPE'],
      dtype='object', length=110)

Index(['SEGID', 'BMP', 'EMP', 'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID',
       'CO_FIPS', 'CCSGROUP19', 'FACMANADJ',
       ...
       'CUTRK2014', 'SUTRK2013', 'CUTRK2013', 'SUTRK2012', 'CUTRK2012',
       'SUTRK2011', 'CUTRK2011', 'SUTRK2010', 'CUTRK2010', 'SHAPE'],
      dtype='object', length=110)

In [145]:
sdfForecastsPlusSegments = pd.merge(sdfForecasts,sdfSegments,on='SEGID',how='right')
sdfForecastsPlusSegments

,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,CUTRK2014,SUTRK2013,CUTRK2013,SUTRK2012,CUTRK2012,SUTRK2011,CUTRK2011,SUTRK2010,CUTRK2010,SHAPE
0,0023_000.0,2000.0,2400.0,2600.0,2700.0,3100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(422233.4001000002, 4607564.0001),..."
1,0023_001.6,2900.0,3200.0,3400.0,3300.0,3500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(422182.7999999998, 4610094.1), (4..."
2,0023_002.1,2900.0,3400.0,3500.0,3600.0,4100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(422217.0, 4611027.0), (422220.400..."
3,0023_002.5,2200.0,2600.0,2700.0,2700.0,3200.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(421790.7599999998, 4611304.15), (..."
4,0023_004.2,2200.0,2500.0,2700.0,2700.0,3100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'paths': [[(420610.13999999966, 4613651.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,"{'paths': [[(424308.5, 4489341.199999999), (42..."
5834,WFRC_8473,6400.0,6400.0,6800.0,7500.0,8300.0,500.0,0.0,0.0,0.0,...,0,0,0,0.0,0,0,0,0,0,"{'paths': [[(417141.7167999996, 4545904.8543),..."
5835,WFRC_8474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0.0,0,0,0,0,0,"{'paths': [[(422745.79000000004, 4480897.97000..."
5836,WFRC_8475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0.0,0,0,0,0,0,"{'paths': [[(422006.9967, 4481267.654200001), ..."


In [146]:
# cacluate and adjusted base year
sdfForecastsPlusSegments['F2019'] = sdfForecastsPlusSegments['AADT2019'] + sdfForecastsPlusSegments['ADJHIST']

In [147]:
# calculate change fields
periods = [
    ('19TO50', 'F2019', 'F2050'),
    ('19TO23', 'F2019', 'F2023'),
    ('23TO28', 'F2023', 'F2028'),
    ('28TO32', 'F2028', 'F2032'),
    ('32TO42', 'F2032', 'F2042'),
    ('42TO50', 'F2042', 'F2050')
]

for period, start_year_key, end_year_key in periods:
    ch_key = f'CH{period}'
    sdfForecastsPlusSegments[ch_key] = sdfForecastsPlusSegments[end_year_key] - sdfForecastsPlusSegments[start_year_key]

sdfForecastsPlusSegments


,SEGID,MF2023,MF2028,MF2032,MF2042,MF2050,ADJ2023,ADJ2028,ADJ2032,ADJ2042,...,SUTRK2010,CUTRK2010,SHAPE,F2019,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0023_000.0,2000.0,2400.0,2600.0,2700.0,3100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"{'paths': [[(422233.4001000002, 4607564.0001),...",1969.0,1131.0,31.0,400.0,200.0,100.0,400.0
1,0023_001.6,2900.0,3200.0,3400.0,3300.0,3500.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"{'paths': [[(422182.7999999998, 4610094.1), (4...",2955.0,545.0,-55.0,300.0,200.0,-100.0,200.0
2,0023_002.1,2900.0,3400.0,3500.0,3600.0,4100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"{'paths': [[(422217.0, 4611027.0), (422220.400...",2955.0,1145.0,-55.0,500.0,100.0,100.0,500.0
3,0023_002.5,2200.0,2600.0,2700.0,2700.0,3200.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"{'paths': [[(421790.7599999998, 4611304.15), (...",2209.0,991.0,-9.0,400.0,100.0,0.0,500.0
4,0023_004.2,2200.0,2500.0,2700.0,2700.0,3100.0,0.0,0.0,0.0,0.0,...,NaN,NaN,"{'paths': [[(420610.13999999966, 4613651.36999...",2209.0,891.0,-9.0,300.0,200.0,0.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,"{'paths': [[(424308.5, 4489341.199999999), (42...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
5834,WFRC_8473,6400.0,6400.0,6800.0,7500.0,8300.0,500.0,0.0,0.0,0.0,...,0,0,"{'paths': [[(417141.7167999996, 4545904.8543),...",0.0,8300.0,6900.0,-500.0,400.0,700.0,800.0
5835,WFRC_8474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,"{'paths': [[(422745.79000000004, 4480897.97000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5836,WFRC_8475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,"{'paths': [[(422006.9967, 4481267.654200001), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [148]:
lstFieldsFromForecasts

['SEGID',
 'MF2023',
 'MF2028',
 'MF2032',
 'MF2042',
 'MF2050',
 'ADJ2023',
 'ADJ2028',
 'ADJ2032',
 'ADJ2042',
 'ADJ2050',
 'ADJHIST']

In [149]:
lstDropFields = [item for item in lstFieldsFromForecasts if item != 'SEGID']
sdfForecastsPlusSegments.drop(columns=lstDropFields, inplace=True)
sdfForecastsPlusSegments

,SEGID,F2023,F2028,F2032,F2042,F2050,BMP,EMP,DISTANCE,PLANAREA,...,SUTRK2010,CUTRK2010,SHAPE,F2019,CH19TO50,CH19TO23,CH23TO28,CH28TO32,CH32TO42,CH42TO50
0,0023_000.0,2000.0,2400.0,2600.0,2700.0,3100.0,0.000,1.564,1.585102,Cache,...,NaN,NaN,"{'paths': [[(422233.4001000002, 4607564.0001),...",1969.0,1131.0,31.0,400.0,200.0,100.0,400.0
1,0023_001.6,2900.0,3200.0,3400.0,3300.0,3500.0,1.564,2.146,0.580084,Cache,...,NaN,NaN,"{'paths': [[(422182.7999999998, 4610094.1), (4...",2955.0,545.0,-55.0,300.0,200.0,-100.0,200.0
2,0023_002.1,2900.0,3400.0,3500.0,3600.0,4100.0,2.146,2.533,0.384494,Cache,...,NaN,NaN,"{'paths': [[(422217.0, 4611027.0), (422220.400...",2955.0,1145.0,-55.0,500.0,100.0,100.0,500.0
3,0023_002.5,2200.0,2600.0,2700.0,2700.0,3200.0,2.533,4.170,1.633041,Cache,...,NaN,NaN,"{'paths': [[(421790.7599999998, 4611304.15), (...",2209.0,991.0,-9.0,400.0,100.0,0.0,500.0
4,0023_004.2,2200.0,2500.0,2700.0,2700.0,3100.0,4.170,6.289,2.114815,UDOT,...,NaN,NaN,"{'paths': [[(420610.13999999966, 4613651.36999...",2209.0,891.0,-9.0,300.0,200.0,0.0,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.253734,WFRC,...,0,0,"{'paths': [[(424308.5, 4489341.199999999), (42...",0.0,0.0,0.0,0.0,0.0,0.0,0.0
5834,WFRC_8473,6900.0,6400.0,6800.0,7500.0,8300.0,0.000,0.000,0.714015,WFRC,...,0,0,"{'paths': [[(417141.7167999996, 4545904.8543),...",0.0,8300.0,6900.0,-500.0,400.0,700.0,800.0
5835,WFRC_8474,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.232025,WFRC,...,0,0,"{'paths': [[(422745.79000000004, 4480897.97000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5836,WFRC_8475,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.587918,WFRC,...,0,0,"{'paths': [[(422006.9967, 4481267.654200001), ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
sdfForecastsPlusSegments.columns


# Set display options to ensure no truncation
pd.set_option('display.max_columns', None)  # Ensures all columns are shown
pd.set_option('display.max_seq_items', None)  # For potentially long sequences

# Now, when you print or display the DataFrame, all column names will be shown
print(sdfForecastsPlusSegments.columns)

# Reset display options to default
pd.reset_option('display.max_columns')
pd.reset_option('display.max_seq_items')


Index(['SEGID', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050', 'BMP', 'EMP',
       'DISTANCE', 'PLANAREA', 'AADT2019', 'SUBAREAID', 'CO_FIPS',
       'CCSGROUP19', 'FACMANADJ', 'FAC_APR', 'FAC_AUG', 'FAC_DEC', 'FAC_FAL',
       'FAC_FEB', 'FAC_FRI', 'FAC_JAN', 'FAC_JUL', 'FAC_JUN', 'FAC_MAR',
       'FAC_MAX', 'FAC_MAXMO', 'FAC_MAY', 'FAC_MON', 'FAC_NOV', 'FAC_OCT',
       'FAC_SAT', 'FAC_SEP', 'FAC_SPR', 'FAC_SUM', 'FAC_SUN', 'FAC_THU',
       'FAC_TUE', 'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WED', 'FAC_WEMAX', 'FAC_WIN',
       'ROUTE', 'Id_x', 'Id_y', 'AADTSTN', 'AADT2022', 'AADT2021', 'AADT2020',
       'AADT2018', 'AADT2017', 'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013',
       'AADT2012', 'AADT2011', 'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007',
       'AADT2006', 'AADT2005', 'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001',
       'AADT2000', 'AADT1999', 'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995',
       'AADT1994', 'AADT1993', 'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989',
      

In [151]:
lstFinalCols = ['SEGID',
                'ROUTE', 'BMP', 'EMP', 'DISTANCE', 'CO_FIPS', 'SUBAREAID', 'PLANAREA',
                'F2019', 'F2023', 'F2028', 'F2032', 'F2042', 'F2050',
                'CH19TO50', 'CH19TO23', 'CH23TO28', 'CH28TO32', 'CH32TO42', 'CH42TO50', 
                'AADTSTN',
                'AADT2022', 'AADT2021', 'AADT2020', 'AADT2019', 'AADT2018', 'AADT2017',
                'AADT2016', 'AADT2015', 'AADT2014', 'AADT2013', 'AADT2012', 'AADT2011',
                'AADT2010', 'AADT2009', 'AADT2008', 'AADT2007', 'AADT2006', 'AADT2005',
                'AADT2004', 'AADT2003', 'AADT2002', 'AADT2001', 'AADT2000', 'AADT1999',
                'AADT1998', 'AADT1997', 'AADT1996', 'AADT1995', 'AADT1994', 'AADT1993',
                'AADT1992', 'AADT1991', 'AADT1990', 'AADT1989', 'AADT1988', 'AADT1987',
                'AADT1986', 'AADT1985', 'AADT1984', 'AADT1983', 'AADT1982', 'AADT1981',
                'SUTRK2022', 'CUTRK2022', 'SUTRK2021', 'CUTRK2021', 'SUTRK2020',
                'CUTRK2020', 'SUTRK2019', 'CUTRK2019', 'SUTRK2018', 'CUTRK2018',
                'SUTRK2017', 'CUTRK2017', 'SUTRK2016', 'CUTRK2016', 'SUTRK2015',
                'CUTRK2015', 'SUTRK2014', 'CUTRK2014', 'SUTRK2013', 'CUTRK2013',
                'SUTRK2012', 'CUTRK2012', 'SUTRK2011', 'CUTRK2011', 'SUTRK2010',
                'CUTRK2010', 'CCSGROUP19',
                'FAC_MON', 'FAC_TUE', 'FAC_WED', 'FAC_THU', 'FAC_FRI', 'FAC_SAT', 'FAC_SUN',
                'FAC_WDAVG', 'FAC_WEAVG', 'FAC_WEMAX',
                'FAC_SPR', 'FAC_SUM', 'FAC_FAL', 'FAC_WIN', 
                'FAC_JAN', 'FAC_FEB', 'FAC_MAR', 'FAC_APR', 'FAC_MAY', 'FAC_JUN',
                'FAC_JUL', 'FAC_AUG', 'FAC_SEP', 'FAC_OCT', 'FAC_NOV', 'FAC_DEC',
                'FAC_MAXMO','FAC_MAX',  
                'FACMANADJ', 'SHAPE']


sdfForecastsPlusSegmentsFinal = sdfForecastsPlusSegments[lstFinalCols].copy()

# For numeric columns
numeric_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['int', 'float']).columns
sdfForecastsPlusSegmentsFinal[numeric_cols] = sdfForecastsPlusSegmentsFinal[numeric_cols].fillna(0)

# For string/object columns
string_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['object']).columns
sdfForecastsPlusSegmentsFinal[string_cols] = sdfForecastsPlusSegmentsFinal[string_cols].fillna('')

# If you also have categorical columns and want to treat them as strings:
categorical_cols = sdfForecastsPlusSegmentsFinal.select_dtypes(include=['category']).columns
if not categorical_cols.empty:  # Check if there are any categorical columns
    sdfForecastsPlusSegmentsFinal[categorical_cols] = sdfForecastsPlusSegmentsFinal[categorical_cols].fillna('')

sdfForecastsPlusSegmentsFinal

,SEGID,ROUTE,BMP,EMP,DISTANCE,CO_FIPS,SUBAREAID,PLANAREA,F2019,F2023,...,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_MAXMO,FAC_MAX,FACMANADJ,SHAPE
0,0023_000.0,0023,0.000,1.564,1.585102,5.0,2.0,Cache,1969.0,2000.0,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{'paths': [[(422233.4001000002, 4607564.0001),..."
1,0023_001.6,0023,1.564,2.146,0.580084,5.0,2.0,Cache,2955.0,2900.0,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{'paths': [[(422182.7999999998, 4610094.1), (4..."
2,0023_002.1,0023,2.146,2.533,0.384494,5.0,2.0,Cache,2955.0,2900.0,...,1.0366,1.0724,1.0607,1.0468,0.9896,0.9399,8,1.0724,0.0,"{'paths': [[(422217.0, 4611027.0), (422220.400..."
3,0023_002.5,0023,2.533,4.170,1.633041,5.0,2.0,Cache,2209.0,2200.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[(421790.7599999998, 4611304.15), (..."
4,0023_004.2,0023,4.170,6.289,2.114815,5.0,2.0,UDOT,2209.0,2200.0,...,1.0796,1.0897,1.0752,1.0684,0.9511,0.8822,5,1.1065,0.0,"{'paths': [[(420610.13999999966, 4613651.36999..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5833,WFRC_8472,WFRC,0.000,0.000,0.253734,35.0,1.0,WFRC,0.0,0.0,...,0.9960,1.0223,1.0238,1.0103,0.9944,0.9907,5,1.0245,0.0,"{'paths': [[(424308.5, 4489341.199999999), (42..."
5834,WFRC_8473,WFRC,0.000,0.000,0.714015,11.0,1.0,WFRC,0.0,6900.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[(417141.7167999996, 4545904.8543),..."
5835,WFRC_8474,WFRC,0.000,0.000,0.232025,35.0,1.0,WFRC,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[(422745.79000000004, 4480897.97000..."
5836,WFRC_8475,WFRC,0.000,0.000,0.587918,35.0,1.0,WFRC,0.0,0.0,...,1.0100,1.0396,1.0434,1.0358,0.9936,0.9616,9,1.0434,0.0,"{'paths': [[(422006.9967, 4481267.654200001), ..."


In [152]:
sdfForecastsPlusSegmentsFinal.spatial.to_featureclass('results/Traffic-Volume-Historic-and-Forecast.shp',sanitize_columns=False)

'e:\\GitHub\\Traffic-Volume-Forecasts\\results\\Traffic-Volume-Historic-and-Forecast.shp'